# ห้องปฏิบัติการเสริม - การจำแนกประเภทหลายคลาส

## 1.1 เป้าหมาย

ในห้องปฏิบัติการนี้ คุณจะสำรวจตัวอย่างของการจำแนกประเภทหลายคลาสโดยใช้เครือข่ายประสาทเทียม
<figure>
 <img src="./images/C2_W2_mclass_header_v1.png"   style="width500px;height:200px;">
</figure>


## 1.2 เครื่องมือ
คุณจะใช้ฟังก์ชันการพล็อตข้อมูลบางอย่าง ฟังก์ชันเหล่านี้ถูกเก็บไว้ในไฟล์  `lab_utils_multiclass_TF.py` ในไดเรกทอรี่นี้

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
try:
  %matplotlib widget
  print("widget is already installed")
except:
  print("widget is not been installed, install now..")
  !pip install ipympl

In [ ]:

import requests
from pathlib import Path

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/deeplearning.mplstyle'
url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_utils_common_C5.py'
url3 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/autils.py'
url4 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_utils_multiclass_TF.py'

response = requests.get(url)
with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)

response = requests.get(url2)
with open('lab_utils_common.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url3)
with open('autils.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url4)
with open('lab_utils_multiclass_TF.py', 'wb') as f:
  f.write(response.content)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
np.set_printoptions(precision=2)
from lab_utils_multiclass_TF import *
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

# 2.0 การจำแนกประเภทหลายคลาส
เครือข่ายประสาทเทียมมักใช้ในการจำแนกข้อมูล ตัวอย่างของเครือข่ายประสาทเทียมคือ:
- รับภาพถ่ายและจำแนกประเภทของวัตถุในภาพเป็น {สุนัข, แมว, ม้า, อื่น ๆ}
- รับประโยคและจำแนก "ส่วนของคำพูด" ขององค์ประกอบ: {คำนาม, กริยา, คำคุณศัพท์ ฯลฯ} 

เครือข่ายประเภทนี้จะมีหน่วยหลายหน่วยในเลเยอร์สุดท้าย ผลลัพธ์แต่ละรายการเชื่อมโยงกับหมวดหมู่ เมื่อนำตัวอย่างอินพุตไปใช้กับเครือข่าย ผลลัพธ์ที่มีค่าสูงสุดคือหมวดหมู่ที่ทำนาย หากผลลัพธ์ถูกนำไปใช้กับฟังก์ชันซอฟต์แม็กซ์ ผลลัพธ์ของซอฟต์แม็กซ์จะให้ความน่าจะเป็นของอินพุตที่อยู่ในแต่ละหมวดหมู่

ในห้องปฏิบัติการนี้ คุณจะเห็นตัวอย่างของการสร้างเครือข่ายหลายคลาสใน TensorFlow จากนั้นเราจะมาดูวิธีที่เครือข่ายประสาทเทียมทำการทำนาย

มาเริ่มกันด้วยการสร้างชุดข้อมูลสี่คลาส

## 2.1 เตรียมและแสดงข้อมูลของเรา

เราจะใช้ฟังก์ชัน `make_blobs` ของ Scikit-Learn เพื่อสร้างชุดข้อมูลการฝึกที่มี 4 หมวดหมู่ตามที่แสดงในพล็อตด้านล่าง

In [ ]:
# make 4-class dataset for classification
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(n_samples=m, centers=centers, cluster_std=std,random_state=30)

In [ ]:
plt_mc(X_train,y_train,classes, centers, std=std)

จุดแต่ละจุดแทนตัวอย่างการฝึก อักษ์ (x0,x1) คืออินพุตและสีแทนคลาสที่ตัวอย่างนั้นเกี่ยวข้อง เมื่อได้รับการฝึกฝนแล้ว โมเดลจะถูกนำเสนอด้วยตัวอย่างใหม่ (x0,x1) และจะทำนายคลาส

แม้ว่าจะถูกสร้างขึ้น แต่ชุดข้อมูลนี้เป็นตัวแทนของปัญหาการจำแนกประเภทในโลกจริงหลายประเภท มีคุณสมบัติอินพุตหลายอย่าง (x0,...,xn) และหลายหมวดหมู่เอาต์พุต โมเดลได้รับการฝึกฝนเพื่อใช้คุณสมบัติอินพุตในการทำนายหมวดหมู่เอาต์พุตที่ถูกต้อง

In [ ]:
# show classes in data set
print(f"unique classes {np.unique(y_train)}")
# show how classes are represented
print(f"class representation {y_train[:10]}")
# show shapes of our dataset
print(f"shape of X_train: {X_train.shape}, shape of y_train: {y_train.shape}")

## 2.2 Model
<img align="Right" src="./images/C2_W2_mclass_lab_network.PNG"  style=" width:350px; padding: 10px 20px ; ">
ห้องปฏิบัติการนี้จะใช้เครือข่าย 2 ชั้นดังที่แสดงไว้ ไม่เหมือนกับเครือข่ายการจำแนกประเภทแบบไบนารี เครือข่ายนี้มีสี่เอาต์พุต หนึ่งสำหรับแต่ละคลาส เมื่อได้รับตัวอย่างอินพุต เอาต์พุตที่มีค่าสูงสุดคือคลาสที่ทำนายได้ของอินพุต

ด้านล่างนี้เป็นตัวอย่างของวิธีสร้างเครือข่ายนี้ใน TensorFlow โปรดสังเกตว่าเลเยอร์เอาต์พุตใช้การกระตุ้นแบบ `linear` แทนที่จะเป็น `softmax` แม้ว่าจะเป็นไปได้ที่จะรวม softmax ในเลเยอร์เอาต์พุต แต่จะเสถียรทางตัวเลขมากกว่าหากส่งเอาต์พุตแบบ linear ไปยังฟังก์ชันการสูญเสียระหว่างการฝึก หากโมเดลถูกใช้เพื่อทำนายความน่าจะเป็น ซอฟต์แม็กซ์สามารถนำไปใช้ได้ในขณะนั้น

In [ ]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(2, activation = 'relu',   name = "L1"),
        Dense(4, activation = 'linear', name = "L2")
    ]
)

ข้อความต่อไปนี้รวบรวมและฝึกอบรมเครือข่าย การตั้งค่า `from_logits=True` เป็นอาร์กิวเมนต์สำหรับฟังก์ชันการสูญเสียระบุว่าการเปิดใช้งานเอาต์พุตเป็นเชิงเส้นมากกว่าซอฟต์แม็กซ์

In [ ]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

model.fit(
    X_train,y_train,
    epochs=200
)

เมื่อมีการฝึกโมเดลแล้ว เราสามารถดูได้ว่าโมเดลได้จำแนกข้อมูลการฝึกอย่างไร

In [ ]:
plt_cat_mc(X_train, y_train, model, classes)

ด้านบน เส้นแบ่งเขตการตัดสินใจแสดงให้เห็นว่าโมเดลได้แบ่งพื้นที่อินพุตอย่างไร โมเดลที่ง่ายมากนี้ไม่มีปัญหาในการจำแนกข้อมูลการฝึก มันทำได้อย่างไร มาดูเครือข่ายอย่างละเอียดมากขึ้น ด้านล่าง เราจะดึงน้ำหนักที่ได้รับการฝึกอบรมจากโมเดลและใช้เพื่อพล็อตฟังก์ชันของแต่ละหน่วยของเครือข่าย 

ต่อไปจะมีคำอธิบายที่ละเอียดมากขึ้นเกี่ยวกับผลลัพธ์ คุณไม่จำเป็นต้องรู้รายละเอียดเหล่านี้เพื่อใช้เครือข่ายประสาทเทียมได้สำเร็จ แต่การได้รับสัญชาตญาณเพิ่มเติมเกี่ยวกับวิธีที่เลเยอร์รวมกันเพื่อแก้ปัญหาการจำแนกประเภทอาจเป็นประโยชน์

In [ ]:
# gather the trained parameters from the first layer
l1 = model.get_layer("L1")
W1,b1 = l1.get_weights()

In [ ]:
# plot the function of the first layer
plt_layer_relu(X_train, y_train.reshape(-1,), W1, b1, classes)

In [ ]:
# gather the trained parameters from the output layer
l2 = model.get_layer("L2")
W2, b2 = l2.get_weights()
# create the 'new features', the training examples after L1 transformation
Xl2 = np.maximum(0, np.dot(X_train,W1) + b1)

plt_output_layer_linear(Xl2, y_train.reshape(-1,), W2, b2, classes,
                        x0_rng = (-0.25,np.amax(Xl2[:,0])), x1_rng = (-0.25,np.amax(Xl2[:,1])))

## Explanation
#### Layer 1 <img align="Right" src="./images/C2_W2_mclass_layer1.png"  style=" width:600px; padding: 10px 20px ; ">
พล็อตเหล่านี้แสดงฟังก์ชันของหน่วย 0 และ 1 ในเลเยอร์แรกของเครือข่าย อินพุตคือ  ($x_0,x_1$) บนแกน ผลลัพธ์ของหน่วยจะแสดงด้วยสีของพื้นหลัง ซึ่งระบุโดยแถบสีทางด้านขวาของแต่ละกราฟ โปรดสังเกตว่าเนื่องจากหน่วยเหล่านี้ใช้ ReLu ผลลัพธ์จึงไม่จำเป็นต้องอยู่ระหว่าง 0 ถึง 1 และในกรณีนี้มีค่ามากกว่า 20 ที่จุดสูงสุด เส้นโค้งในกราฟนี้แสดงจุดเปลี่ยนระหว่างผลลัพธ์ $a^{[1]}_j$  ที่เป็นศูนย์และไม่ใช่ศูนย์ จำได้ไหมว่ากราฟของ ReLu เป็นอย่างไร :<img align="right" src="./images/C2_W2_mclass_relu.png"  style=" width:200px; padding: 10px 20px ; "> เส้นโค้งในกราฟคือจุดเปลี่ยนของ ReLU

ยูนิต 0 ได้แยกคลาส 0 และ 1 ออกจากคลาส 2 และ 3 จุดทางซ้ายของเส้น (คลาส 0 และ 1) จะส่งออกศูนย์ ในขณะที่จุดทางด้านขวาจะส่งออกค่าที่มากกว่าศูนย์

ยูนิต 1 ได้แยกคลาส 0 และ 2 ออกจากคลาส 1 และ 3 จุดด้านบนของเส้น (คลาส 0 และ 2) จะส่งออกศูนย์ ในขณะที่จุดด้านล่างจะส่งออกค่าที่มากกว่าศูนย์ มาดูกันว่าสิ่งนี้จะทำงานอย่างไรในเลเยอร์ถัดไป!

#### Layer 2, the output layer  <img align="Right" src="./images/C2_W2_mclass_layer2.png"  style=" width:600px; padding: 10px 20px ; ">

**"จุดในกราฟเหล่านี้คือตัวอย่างการฝึกอบรมที่แปลโดยเลเยอร์แรก หนึ่งในวิธีคิดเกี่ยวกับเรื่องนี้คือเลเยอร์แรกได้สร้างชุดคุณลักษณะใหม่สำหรับการประเมินโดยเลเยอร์ที่ 2 แกนในพล็อตเหล่านี้คือผลลัพธ์ของเลเยอร์ก่อนหน้า  $a^{[1]}_0$ และ  $a^{[1]}_1$. ตามที่คาดการณ์ไว้ข้างต้น คลาส 0 และ 1 (สีน้ำเงินและสีเขียว) มี  $a^{[1]}_0 = 0$ ในขณะที่คลาส 0 และ 2 (สีน้ำเงินและสีส้ม) มี   $a^{[1]}_1 = 0$.  
อีกครั้ง ความเข้มของสีพื้นหลังบ่งชี้ค่าสูงสุด หน่วย 0 จะสร้างค่าสูงสุดสำหรับค่าใกล้ (0,0) ซึ่งคลาส 0 (สีน้ำเงิน) ได้รับการแมป หน่วย 1 สร้างค่าสูงสุดในมุมซ้ายบนเลือกคลาส 1 (สีเขียว) หน่วย 2 เลือกมุมล่างขวาซึ่งคลาส 2 (สีส้ม) อยู่ หน่วย 3 สร้างค่าสูงสุดในมุมขวาบนเลือกคลาสสุดท้ายของเรา (สีม่วง)

อีกแงหนึ่งที่ไม่ชัดเจนจากกราฟคือค่าเหล่านี้ได้รับการประสานงานระหว่างหน่วย ไม่เพียงพอที่หน่วยจะสร้างค่าสูงสุดสำหรับคลาสที่เลือกเท่านั้น แต่ยังต้องเป็นค่าสูงสุดของหน่วยทั้งหมดสำหรับจุดในคลาสนั้นด้วย สิ่งนี้ทำได้โดยฟังก์ชันซอฟต์แม็กซ์โดยนัยที่เป็นส่วนหนึ่งของฟังก์ชันการสูญเสีย (`SparseCategoricalCrossEntropy`) ไม่เหมือนกับฟังก์ชันการกระตุ้นอื่น ๆ ซอฟต์แม็กซ์ทำงานข้ามเอาต์พุตทั้งหมด

คุณสามารถใช้เครือข่ายประสาทเทียมได้สำเร็จโดยไม่รู้รายละเอียดของสิ่งที่แต่ละหน่วยกำลังทำหวังว่าตัวอย่างนี้จะให้ความรู้สึกเกี่ยวกับสิ่งที่เกิดขึ้นภายใต้ฮูด

## ขอแสดงความยินดี!

คุณได้เรียนรู้วิธีการสร้างและใช้งานเครือข่ายประสาทเทียมสำหรับการจำแนกประเภทหลายคลาสแล้ว
